In [5]:
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
!npm install -g svelte-cli@2.2.0

▄ ╢░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
▀ ╢░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
▐ ╢░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
WARN engine svelte-cli@2.2.0: wanted: {"node":">=6.0.0"} (current: {"node":"4.2.6","npm":"3.5.2"})
WARN engine svelte-cli@2.2.0: wanted: {"node":">=6.0.0"} (current: {"node":"4.2.6","npm":"3.5.2"})
loadDep:svelte → request  ▌ ╢██████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
loadDep:svelte → get      ▀ ╢██████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
loadDep:svelte → afterAdd ▀ ╢██████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
npm WARN checkPermissions Missing write access to /usr/local/lib/node_modules/svelte-cli
diffTrees                 ▀ ╢█████████████████████████████░░░░░░░░░░░░░░░░░░░░╟
npm WARN checkPermissions Missing write access to /usr/local/lib/node_modules
diffTrees                 ▐ ╢█████████████████████████████░░░░░░░░░░░░░░░░░

In [1]:
import random
import json
import tempfile
import subprocess
import os.path as osp
import uuid

from IPython.core.magic import register_cell_magic

from lucid.misc.io.showing import _display_html
from lucid.misc.io.reading import read

_svelte_temp_dir = tempfile.mkdtemp(prefix="svelte_")

_template = """
  <div id='$id'></div>
  <script>
  $js
  </script>
  <script>
    var app = new $name({
        target: document.querySelector('#$id'),
        data: $data,
      });
  </script>
 """


def build_svelte(html_fname):
  js_fname = html_fname.replace(".html", ".js")
  cmd = "svelte compile --format iife " + html_fname + " > " + js_fname
  print(cmd)
  try:
    print(subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT))
  except subprocess.CalledProcessError as exception:
    print("Svelte build failed! Output:\n{}".format(exception.output.decode()))
  return js_fname


def SvelteComponent(name, path):
  """Display svelte components in iPython.
  Args:
    name: name of svelte component (must match component filename when built)
    path: path to compile svelte .js file or source svelte .html file.
      (If html file, we try to call svelte and build the file.)
  Returns:
    A function mapping data to a rendered svelte component in ipython.
  """
  if path[-3:] == ".js":
    js_path = path
  elif path[-5:] == ".html":
    print("Trying to build svelte component from html...")
    js_path = build_svelte(path)
  js_content = read(js_path)
  def inner(data):
    id_str = name + "_" + str(uuid.uuid4())
    html = _template \
        .replace("$js", js_content) \
        .replace("$name", name) \
        .replace("$data", json.dumps(data)) \
        .replace("$id", id_str)
    _display_html(html)
  return inner


@register_cell_magic
def html_define_svelte(line, cell):
  base_name = line.split()[0]
  # name_str will become the name of a javascript variable, and can't contain dashes.
  name_str = base_name + "_" + str(uuid.uuid4()).replace('-', '_')
  html_fname = osp.join(_svelte_temp_dir, name_str + ".html")
  with open(html_fname, "w") as f:
    f.write(cell)
  component = SvelteComponent(name_str, html_fname)
  globals()[base_name] = component

In [2]:
%%html_define_svelte GroupWidget

<div class="figure" style="width: 600px;">
  <div class="outer" on:mouseover="set({pres_n: undefined})">
    <img src="{{img}}">
    {{#if pres_n != undefined}}
    <img src="{{spatial_factors[pres_n]}}" class="overlay"  >
    {{/if}}
  </div>
   
  <div class="outer" on:mouseover="set({pres_n: undefined})">
    <div style="width:100%; height: 100%; background-color: #000;"></div>
    {{#each range(n_groups) as n}}
    {{#if pres_n == undefined || pres_n == n}}
    <img src="{{spatial_factors[n]}}" class="factor" 
         style="filter: hue-rotate({{n*360/n_groups}}deg);">
    {{/if}}
    {{/each}}
  </div>
  
  <br>
  <br>
  
  <div on:mouseleave="set({pres_n: undefined})">
    {{#each range(n_groups) as n}}
    <div class="group" style="background-color: hsl({{n*360/n_groups}}, 80%, 50%); "
         on:mouseover="set({pres_n: n})">
      <img src="{{group_icons[n]}}">
    </div>
    {{/each}}
  </div>
  
</div>


<style>
  .outer{
    width: 224px;
    height: 224px;
    display: inline-block;
    margin-right: 2px;
    position: relative;
  }
  .outer img {
    position: absolute;
    left: 0px;
    top: 0px;
    width: 224px;
    height: 224px;
    image-rendering: pixelated; 
  }
  .factor {
    mix-blend-mode: lighten;
  }
  .overlay {
    filter: grayscale(100%) brightness(3);
    mix-blend-mode: multiply;
  }
  .group {
    width: 80px;
    height: 80px;
    margin-right: 4px;
    display: inline-block;
    padding-top: 10px;
  }
</style>

<script>
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        img: "",
        n_groups: 1,
        spatial_factors: [""],
        group_icons: [""],
        pres_n: undefined,
        
      };
    },
    helpers: {range}
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_vN1QOp/GroupWidget_72f63987_642c_47d4_8f16_454c33c687fc.html > /tmp/svelte_vN1QOp/GroupWidget_72f63987_642c_47d4_8f16_454c33c687fc.js
Svelte build failed! Output:
/usr/local/lib/node_modules/svelte-cli/bin.js:2155
			const [key, value] = pair.split(':');
			      ^

SyntaxError: Unexpected token [
    at exports.runInThisContext (vm.js:53:16)
    at Module._compile (module.js:374:25)
    at Object.Module._extensions..js (module.js:417:10)
    at Module.load (module.js:344:32)
    at Function.Module._load (module.js:301:12)
    at Function.Module.runMain (module.js:442:10)
    at startup (node.js:136:18)
    at node.js:966:3



In [4]:

!svelte compile --format iife /tmp/svelte_FQAAcC/GroupWidget_bcdc5edb_94d8_446c_b343_6ecf0b1ea6aa.html > /tmp/svelte_FQAAcC/GroupWidget_bcdc5edb_94d8_446c_b343_6ecf0b1ea6aa.js

/usr/bin/env: ‘node’: No such file or directory
